# Handling duplicate, missing, or invalid data

## About the Data
In this notebook, we will be working with FIFA players data for 2022 obtained from [Kaggle](https://www.kaggle.com/datasets/stefanoleone992/fifa-22-complete-player-dataset)

## Setup
We will be working with the `players_22.csv` file, so we need to handle our imports and read it in.

In [1]:
import pandas as pd

In [2]:
players = pd.read_csv(
    'dirty_data.csv', 
    usecols=['short_name', 'wage_eur', 'age', 'club_name', 'nationality_name', 'preferred_foot']
)

## Finding problematic data
A good first step is to look at some rows:

In [3]:
players.head()

,short_name,wage_eur,age,club_name,nationality_name,preferred_foot
0,L. Messi,320000.0,34,Paris Saint-Germain,Argentina,Left
1,R. Lewandowski,270000.0,32,FC Bayern München,Poland,Right
2,Cristiano Ronaldo,270000.0,36,Manchester United,Portugal,Right
3,Neymar Jr,270000.0,29,Paris Saint-Germain,Brazil,Right
4,K. De Bruyne,350000.0,30,Manchester City,Belgium,Right


Looking at summary statistics can reveal strange or missing values:

In [4]:
players.describe()

,wage_eur,age
count,19204.000000,19265.000000
mean,9006.732972,25.203478
std,19459.399118,4.749543
min,500.000000,16.000000
25%,1000.000000,21.000000
50%,3000.000000,25.000000
75%,8000.000000,29.000000
max,350000.000000,54.000000


The `info()` method can pinpoint missing values and wrong data types:

In [5]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19265 entries, 0 to 19264
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   short_name        19265 non-null  object 
 1   wage_eur          19204 non-null  float64
 2   age               19265 non-null  int64  
 3   club_name         19204 non-null  object 
 4   nationality_name  19265 non-null  object 
 5   preferred_foot    19265 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 903.2+ KB


We can use the `isna()`/`isnull()` method of the series to find nulls:

In [6]:
contain_nulls = players[
    players.short_name.isna() | players.wage_eur.isna() | players.age.isna()
    | players.club_name.isna() | players.nationality_name.isna()
    | players.preferred_foot.isna()
]
contain_nulls.shape[0]

61

In [7]:
contain_nulls.head(10)

,short_name,wage_eur,age,club_name,nationality_name,preferred_foot
147,T. Alderweireld,NaN,32,NaN,Belgium,Right
416,G. Krychowiak,NaN,31,NaN,Poland,Right
694,A. Dzyuba,NaN,32,NaN,Russia,Right
926,M. Ozdoev,NaN,28,NaN,Russia,Right
1146,Y. Zhirkov,NaN,37,NaN,Russia,Left
1152,A. Shunin,NaN,34,NaN,Russia,Left
1182,A. Dragović,NaN,30,NaN,Austria,Right
1241,M. Borjan,NaN,33,NaN,Canada,Right
1255,V. Claesson,NaN,29,NaN,Sweden,Right
1759,D. Kuzyaev,NaN,28,NaN,Russia,Right


We can use the `duplicated()` method to find duplicate rows:

In [8]:
players[players.duplicated()].shape[0]

26

The default for keep is 'first' meaning it won't show the first row that the duplicated data was seen in; we can pass in False to see it:

In [9]:
players[players.duplicated(keep=False)].shape[0]

52

Let's look at a few duplicates:

In [10]:
players[players.duplicated()].head()

,short_name,wage_eur,age,club_name,nationality_name,preferred_foot
19239,Yang Haoyu,1000.0,20,Dalian Professional Football Club,China PR,Right
19240,R. Mahon,500.0,18,Bohemian FC,Republic of Ireland,Left
19241,S. Black,500.0,19,Finn Harps,Republic of Ireland,Right
19242,H. Wiles-Richards,1000.0,19,Bristol City,England,Right
19243,J. Affonso,500.0,23,Cerro Largo Fútbol Club,Uruguay,Right


## Mitigating Issues

### Handling duplicated data
When dealing with duplicate data, we need to think of the ramifications of removing it. If we determine it won't impact our analysis, we can use `drop_duplicates()` to remove them:

In [11]:
pl_clean = players.drop_duplicates()

In [12]:
pl_clean[pl_clean.duplicated()].shape[0]

0

### Dealing with nulls
We could drop nulls, replace them with some arbitrary value, or impute them using the surrounding data. Each of these options may have ramifications, so we must choose wisely.

We can use `dropna()` to drop rows where any column has a null value:

In [13]:
pl_clean.dropna().shape

(19178, 6)

If we pass `how='all'`, we can choose to only drop rows where everything is null, but this removes nothing:

In [14]:
pl_clean.dropna(how='all').shape

(19239, 6)

We can use just a subset of columns to determine what to drop with the `subset` argument:

In [15]:
pl_clean.dropna(
    how='all', subset=['wage_eur', 'club_name']
).shape

(19178, 6)

This can also be performed along columns, and we can also require a certain number of null values before we drop the data:

In [16]:
pl_clean.dropna(axis='columns', thresh=pl_clean.shape[0] * .75).shape

(19239, 6)

We can choose to fill in the null values instead with `fillna()`:

In [17]:
pl_clean.club_name.fillna('unknown', inplace = True)
pl_clean[pl_clean.club_name == 'unknown'].head()

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,short_name,wage_eur,age,club_name,nationality_name,preferred_foot
147,T. Alderweireld,NaN,32,unknown,Belgium,Right
416,G. Krychowiak,NaN,31,unknown,Poland,Right
694,A. Dzyuba,NaN,32,unknown,Russia,Right
926,M. Ozdoev,NaN,28,unknown,Russia,Right
1146,Y. Zhirkov,NaN,37,unknown,Russia,Left


We can couple `fillna()` with other types of calculations. Here we replace missing values of `wage_eur` with the median of all `wage_eur` values:

In [18]:
pl_clean.assign(
    wage_eur = lambda x: x.wage_eur.fillna(x.wage_eur.median())
).loc[[147, 416, 694, 926, 1146], :]

,short_name,wage_eur,age,club_name,nationality_name,preferred_foot
147,T. Alderweireld,3000.0,32,unknown,Belgium,Right
416,G. Krychowiak,3000.0,31,unknown,Poland,Right
694,A. Dzyuba,3000.0,32,unknown,Russia,Right
926,M. Ozdoev,3000.0,28,unknown,Russia,Right
1146,Y. Zhirkov,3000.0,37,unknown,Russia,Left
